In [1]:
import numpy as np
import pandas as pd

# 读取数据
gpa1_data = pd.read_csv("data/gpa1.csv")

# 显示列名
gpa1_data.columns

Index(['age', 'soph', 'junior', 'senior', 'senior5', 'male', 'campus',
       'business', 'engineer', 'colGPA', 'hsGPA', 'ACT', 'job19', 'job20',
       'drive', 'bike', 'walk', 'voluntr', 'PC', 'greek', 'car', 'siblings',
       'bgfriend', 'clubs', 'skipped', 'alcohol', 'gradMI', 'fathcoll',
       'mothcoll'],
      dtype='object')

In [2]:
import statsmodels.api as sm

# 选择自变量和因变量
X = gpa1_data[["hsGPA", "ACT"]]
y = gpa1_data["colGPA"]

# 向自变量矩阵X添加常数项，以便包含截距
X = sm.add_constant(X)

In [3]:
X.head()

,const,hsGPA,ACT
0,1.0,3.0,21
1,1.0,3.2,24
2,1.0,3.6,26
3,1.0,3.5,27
4,1.0,3.9,28


In [4]:
# 创建OLS回归模型的实例并拟合数据
model = sm.OLS(y, X)
results = model.fit()

In [5]:
# 输出回归结果摘要
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 colGPA   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     14.78
Date:                   日, 19 5月 2024   Prob (F-statistic):           1.53e-06
Time:                        23:41:17   Log-Likelihood:                -46.573
No. Observations:                 141   AIC:                             99.15
Df Residuals:                     138   BIC:                             108.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2863      0.341      3.774      0.0

In [18]:
from sklearn.datasets import load_digits

X, _ = load_digits(return_X_y=True)

In [19]:
X

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])